# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [22]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep

In [23]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-project-bank-churn'

experiment=Experiment(ws, experiment_name)

In [27]:
# Quick check to see what datasets are already registerd
for a in ws.datasets.keys():
    print(a)

Bank-churn
Bank-markering
bankchurn2


In [30]:
# get external data https://raw.githubusercontent.com/fuzzballb/nd00333-capstone/master/starter_file/cleaned_data.csv

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Bank-churn"
description_text = "Bank churn DataSet for Udacity Course"

if key in ws.datasets.keys():
        print("using existing dataset") 
        found = True
        dataset = ws.datasets[key] 

if not found:
        print("retrieving dataset") 
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/fuzzballb/nd00333-capstone/master/starter_file/cleaned_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

using existing dataset


KeyboardInterrupt: 

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 30,
                    "max_concurrent_iterations":4,
                    "iterations": 10,
                    "primary_metric" : 'AUC_weighted'                     
}
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target = compute_target,
    task="classification",
    primary_metric="accuracy",
    training_data = dataset,
    label_column_name= "Exited",
    n_cross_validations=5,
    **automl_settings)   

NameError: name 'compute_target' is not defined

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
remote_run

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_automl_run,fitted_model = remote_run.get_output()

In [ ]:
#TODO: Save the best model
import joblib
joblib.dump(value=fitted_model, filename='outputs/automl_test_experiment_best_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model, InferenceConfig
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment.from_conda_specification(name='automlv2',file_path='conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)
service_name = 'bankchurnpredictionautoml'

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4, auth_enabled = True, enable_app_insights = True)
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)
print(service.state)

service.wait_for_deployment(show_output = True)
print(service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json

#CreditScore	Gender	Age	Tenure	Balance	NumOfProducts	HasCrCard	IsActiveMember	EstimatedSalary	[Exited]	Geography_France	Geography_Germany	Geography_Spain

#502	1	42	8	159660.8	3	1	0	113931.57	[1] 1	0	0
#543	0	22	8	0	        2	0	0	127587.22	[0]	1	0	0

data = {"data":
        [
          {
            "CreditScore": 	502,
            "Gender": 1,	
            "Age": 42,	
            "Tenure": 8,	
            "Balance": 159660.8,	
            "NumOfProducts":	3,
            "HasCrCard":	1,
            "IsActiveMember": 0,	
            "EstimatedSalary": 113931.57,	
            "Geography_France": 1,	
            "Geography_Germany": 0,	
            "Geography_Spain": 0
          },
          {
            "CreditScore": 	543,
            "Gender": 0,	
            "Age": 22,	
            "Tenure": 8,	
            "Balance": 0,	
            "NumOfProducts":	2,
            "HasCrCard":	0,
            "IsActiveMember": 0,	
            "EstimatedSalary": 127587.22,	
            "Geography_France": 1,	
            "Geography_Germany": 0,	
            "Geography_Spain": 0
          }
      ]
    }

inputJson = json.dumps(data)
serviceOutput = service.run(inputJson)

print(serviceOutput)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()